In [16]:
#defines

base_path = 'C:/Users/I078212/workspace/plc/hana/'

xsprivileges_files = [
    base_path + 'src/PLC/sap/plc_repl/xs/.xsprivileges',
    base_path + 'src/PLC/sap/plc/doc/.xsprivileges',
    base_path + 'src/PLC/sap/plc/xs/.xsprivileges'
]

print(xsprivileges_files)


"""
src/PLC/sap/plc/doc/.xsprivileges as an example:
{
    "privileges" : [
        { "name" : "API_Documentation_Read", "description": "Read access for the swagger api documentation"}
    ]
}
"""

['C:/Users/I078212/workspace/plc/hana/src/PLC/sap/plc_repl/xs/.xsprivileges', 'C:/Users/I078212/workspace/plc/hana/src/PLC/sap/plc/doc/.xsprivileges', 'C:/Users/I078212/workspace/plc/hana/src/PLC/sap/plc/xs/.xsprivileges']


'\nsrc/PLC/sap/plc/doc/.xsprivileges as an example:\n{\n    "privileges" : [\n        { "name" : "API_Documentation_Read", "description": "Read access for the swagger api documentation"}\n    ]\n}\n'

In [17]:
# parse .xsprivileges which are in json files

import json

in_jsons = []

for f in xsprivileges_files:
    with open(f) as json_data:
        in_jsons.append(json.load(json_data))

print(in_jsons[0])

{'privileges': [{'name': 'generators', 'description': 'Allows usage of artifact generators.'}]}


In [33]:
# generate scopes, role-templates

scopes = []
for data in in_jsons:
    for priviledge in data['privileges']:
        scope = {}
        scope['name'] = '$XSAPPNAME.' + priviledge['name']
        scope['description'] = priviledge['description']
        scopes.append(scope)
#print(scopes)

role_templates = []

# add the most basic role templates, each is associated with one scope
for data in in_jsons:
    for priviledge in data['privileges']:
        role_template = {}
        role_template['name'] = priviledge['name'] + '_RT'
        role_template['scope-references'] = ['$XSAPPNAME.' + priviledge['name']]
        role_templates.append(role_template)


In [34]:
# write output file
xs_security_file = 'C:/Users/I078212/tmp/xs-security.json'

xs_security = {}
xs_security['xsappname'] = 'HCO_PLC'
xs_security['scopes'] = scopes
xs_security['role-templates'] = role_templates
#print(xs_security)

import json
with open(xs_security_file, 'w') as out_file:
    out_file.write(json.dumps(xs_security, ensure_ascii=False, indent=4))
